In [1]:
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel,AsyncOpenAI, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from IPython.display import Markdown, display

In [2]:
model = OpenAIChatCompletionsModel(model="gpt-oss:20b",
                                  openai_client=AsyncOpenAI(base_url="http://localhost:11434/v1",api_key="ollama"))

In [3]:
# Create the agent
agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant",
    model=model
)

In [4]:
result = await Runner.run(starting_agent=agent,input="Who is the father of Mahadev? Answer in short.")
display(Markdown(result.final_output))

OPENAI_API_KEY is not set, skipping trace export


**Father of Mahadev** – Sage **Kashyapa**.

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is no

In [5]:
result = Runner.run_streamed(agent, "I am from India. Create a meal plan for a week. Answer in short.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

**One‑week Indian meal plan (short, balanced)**  

| Day | Breakfast | Lunch | Snack | Dinner |
|-----|-----------|-------|-------|--------|
| **Mon** | Poha + fresh coconut chutney | Chole + 2 chapatis | Makhana (Fox‑nuts) | Khichdi + cucumber raita |
| **Tue** | Upma + mint tea | Mutton curry + steamed rice | Fruit (papaya slices) | Paneer makhani + jeera rice |
| **Wed** | Idli + sambar + coconut chutney | Dal tadka + 2 phulkas | Roasted chickpeas | Fish (grilled/murg) + mixed veg stir‑fry |
| **Thu** | Aloo paratha + curd | Rajma + 2 bajra roti | Boiled sweet corn | Mixed vegetable curry + paratha |
| **Fri** | Banana‑choco pancakes (oats) | Chicken tikka masala + naan | Yogurt smoothie | Tofu tikka + quinoa |
| **Sat** | Dhokla + green chutney | Palak chicken + 2 madhura rotis | Sprouts salad | Biryani (vegetable or lamb) |
| **Sun** | Parle‑Biscuit + chai | Khichdi + mango pickle | Guava slices | Butter chicken + 2 chapatis |

*Adjust portion sizes, add a glass of water or butter

In [30]:
## Traces
import mlflow
import asyncio
from agents import Agent, Runner

mlflow.openai.autolog()

# Optional: Set a tracking URI and an experiment
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("OpenAI Agent")

<Experiment: artifact_location='mlflow-artifacts:/615119549503924654', creation_time=1756996413561, experiment_id='615119549503924654', last_update_time=1756996413561, lifecycle_stage='active', name='OpenAI Agent', tags={}>

In [ ]:

# === 🧠 AGENT: Emoji Generator ===
emoji_agent = Agent(
    name="emoji_agent",
    instructions=(
        "You're an emoji artist. You turn summarized news into expressive emoji sequences. "
        "Be creative, thematic, and intuitive — like visual headlines people can understand without words."
    ),
    model=model
)

emoji_tool = emoji_agent.as_tool(
    tool_name="emoji_generator",
    tool_description="Generates creative emoji representations from short summarized news text."
)

# === ✏️ AGENT: Summarizer ===
summarizer = Agent(
    name="summarizer",
    instructions=(
        "You're a news summarizer. Your job is to shorten long news into 1-2 crisp sentences. "
        "Avoid emojis — keep it short, simple, and informative."
    ),
    model=model
)

summarizer_tool = summarizer.as_tool(
    tool_name="text_summarizer",
    tool_description="Summarizes long text into short, emoji-ready summaries."
)

# === 🎬 AGENT: Orchestrator ===
emoji_headline_generator = Agent(
    name="emoji_headlines",
    instructions=(
        "You're an assistant that turns full news stories into emoji-based headlines. "
        "Use the available tools: first summarize, then convert to emojis. Never create emojis yourself."
    ),
    tools=[summarizer_tool, emoji_tool],
    model=model
)

# === 🧪 TEST DATA ===
news_sample = """
NASA has announced its Artemis III mission will land the first woman and the next man on the Moon by 2026. 
This marks a major milestone in the agency’s goal to establish a long-term human presence on the lunar surface, 
and eventually launch crewed missions to Mars.
"""

# === 🚀 RUN WORKFLOW ===
emoji_response = await Runner.run(emoji_headline_generator, news_sample)
emoji_output = emoji_response.final_output

# === 📤 DISPLAY ===
print("🗞️  Original News:")
print(news_sample)
print("\n✨ Emoji Headline:")
display(Markdown(emoji_output))


In [ ]:

# Define a simple multi-agent workflow
hindi_agent = Agent(
    name="Hindi agent",
    instructions="You only speak Hindi.",
    model=model
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
    model=model
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[hindi_agent, english_agent],
    model=model
)

print('Runner started.')
result = await Runner.run(triage_agent, input="एआई एजेंटों के बड़े में तुम क्या जानते हो?")
print(result.final_output)


In [11]:
import asyncio

from agents import Agent, Runner, function_tool

# Enable auto tracing for OpenAI Agents SDK
mlflow.openai.autolog()


@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."


agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
    model=model
)



result = await Runner.run(agent, input="What's the weather in Tokyo?")
print(result.final_output)

The weather in Tokyo is sunny.


Trace(trace_id=tr-3d9df1cc6f20caa2b4a7d88c964274f3)

In [12]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
)

# Enable auto tracing for OpenAI Agents SDK
mlflow.openai.autolog()


class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str


guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
    model = model
)


@input_guardrail
async def math_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math_homework,
    )


agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],
    model= model
)



try:
    await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
    print("Guardrail didn't trip - this is unexpected")

except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")

MaxTurnsExceeded: Max turns (10) exceeded

Trace(trace_id=tr-a22ed61c9be9c3c5192fcec7a7ccda1e)